In [7]:
import numpy as np
from scipy.optimize import least_squares

# -------------------------------
# 0) Physikalische Konstanten
# -------------------------------
h = 6.62607015e-34      # Planck [J·s]
kB = 1.380649e-23       # Boltzmann [J/K]
c  = 2.99792458e8       # Lichtgeschwindigkeit [m/s]

# -------------------------------
# 1) Hilfsfunktionen
# -------------------------------
def nu_from_lambda(lam_um):
    """Wellenzahl/Frequenz nu [Hz] aus Wellenlänge lam [µm]."""
    lam_m = lam_um * 1e-6
    return c / lam_m

def B_star_nu(T, nu):
    """
    Modifizierte Planck-Funktion 𝓑*ν(T) [W·m^-2·sr^-1·Hz^-1]
    𝓑* = (2 h ν^3 / c^2) * [ exp(hν/kT) - 1 + (hν)^2/(10 (kT)^2) ]^-1
    Hinweis: Das ist eine Frequenz-Darstellung (pro Hz).
    """
    x = (h * nu) / (kB * T)
    denom = np.exp(x) - 1.0 + (x**2) / 10.0
    return (2.0 * h * nu**3) / (c**2) / denom

def psi_lambda(lam_um, osc_const=1e-14):
    """
    Resonanzverstärkung Ψ(λ) = [ sin(2πν·C) / (2πν·C) ]^2   (dimensionslos)
    mit ν = c/λ  und C = 'Materie-Oszillationskonstante' (Standard: 1e-14 s).
    """
    nu = nu_from_lambda(lam_um)
    x  = 2.0 * np.pi * nu * osc_const
    # numerisch stabil: sin(x)/x -> 1 bei x→0
    y = np.ones_like(x)
    nz = np.abs(x) > 1e-12
    y[nz] = (np.sin(x[nz]) / x[nz])**2
    return y

def emissivity_from_Rs(Rs):
    """
    Emissivität ε(λ) = 1 - (2/π) * arcsin( sqrt(Rs) )
    Rs: spekulare Reflexion (z.B. bei 45°), 0..1
    """
    Rs_clamped = np.clip(Rs, 0.0, 1.0)
    return 1.0 - (2.0/np.pi) * np.arcsin(np.sqrt(Rs_clamped))

# -------------------------------
# 2) Eingänge (Beispiel/Placeholder)
# -------------------------------
# Wellenlängenachse [µm]
lam_um = np.linspace(8.0, 12.0, 81)  # LWIR-Fenster Beispiel

# Beispiel: spekulare Reflexion Rs(λ) (falls nicht vorhanden -> anpassen/ermitteln)
# Hier Dummy: leicht materialspezifische Kurve
Rs = 0.05 + 0.02*np.sin(2*np.pi*(lam_um-8.0)/4.0)
eps = emissivity_from_Rs(Rs)           # ε(λ)
rho = 1.0 - eps                        # Reflexionsvermögen

# Umgebungsspektrum X(λ) (optional; auf 0 setzen, wenn Indoor/Eigenemission-only)
X = 1e-6 * (0.6 + 0.4*np.cos(2*np.pi*(lam_um-8.0)/4.0))  # Dummy in "Spektraleinheiten"

# Wahrer Temperaturwert (für Tests/Simulation)
T_true = 300.0  # K

# -------------------------------
# 3) Vorwärtsmodell nach neuer Gleichung
# -------------------------------
def forward_model(T, lam_um, eps, X, use_reflection=True):
    """
    S(λ,T) = ε(λ)·𝓑*ν(T)·Ψ(λ) + (1-ε(λ))·X(λ) [optional]
    Rückgabe in konsistenten (relativen) Einheiten. 
    """
    nu = nu_from_lambda(lam_um)
    Bstar = B_star_nu(T, nu)          # 𝓑*ν(T)
    Psi   = psi_lambda(lam_um)         # Ψ(λ)
    S_emit = eps * Bstar * Psi         # Eigenemission
    if use_reflection:
        S = S_emit + (1.0 - eps) * X   # + Reflexion
    else:
        S = S_emit
    return S

# Synthese "gemessener" Daten
S_emit_true = forward_model(T_true, lam_um, eps, X, use_reflection=False)
S_refl_true = (1.0 - eps) * X
S_meas = S_emit_true + S_refl_true

# Optional: Rauschen hinzufügen
# rng = np.random.default_rng(42)
# S_meas = S_meas * (1.0 + 0.005 * rng.standard_normal(S_meas.shape))

# -------------------------------
# 4) Inversion: T aus S_meas (ε, X gegeben)
# -------------------------------
def residual_T(T_vec):
    T = float(T_vec[0])
    model = forward_model(T, lam_um, eps, X, use_reflection=True)
    return S_meas - model   # Least-Squares-Residuum

res = least_squares(residual_T, x0=[300.0], bounds=(200.0, 500.0), max_nfev=200)
T_est = float(res.x[0])

# -------------------------------
# 5) Ergebnisse
# -------------------------------
print("Wellenlängen [µm]:", np.round(lam_um, 3))
print("Emissivität - eps(λ):", np.round(eps, 4))
print("rho(λ)= (1-eps(λ)):", np.round(rho, 4))
print("X(λ) Umgebung:", np.round(X, 6))
print("S_meas(λ) Komplete Satrhlung mit der Umgebung Strahlung:", np.round(S_meas, 9))
print(f"T_true = {T_true:.2f} K,  T_est = {T_est:.2f} K")
print(f"Fit-Restfehler (Sum of squares) = {res.cost:.3e}")


Wellenlängen [µm]: [ 8.    8.05  8.1   8.15  8.2   8.25  8.3   8.35  8.4   8.45  8.5   8.55
  8.6   8.65  8.7   8.75  8.8   8.85  8.9   8.95  9.    9.05  9.1   9.15
  9.2   9.25  9.3   9.35  9.4   9.45  9.5   9.55  9.6   9.65  9.7   9.75
  9.8   9.85  9.9   9.95 10.   10.05 10.1  10.15 10.2  10.25 10.3  10.35
 10.4  10.45 10.5  10.55 10.6  10.65 10.7  10.75 10.8  10.85 10.9  10.95
 11.   11.05 11.1  11.15 11.2  11.25 11.3  11.35 11.4  11.45 11.5  11.55
 11.6  11.65 11.7  11.75 11.8  11.85 11.9  11.95 12.  ]
Emissivität - eps(λ): [0.8564 0.8542 0.8519 0.8498 0.8477 0.8456 0.8437 0.8419 0.8401 0.8385
 0.837  0.8356 0.8344 0.8332 0.8323 0.8314 0.8308 0.8302 0.8298 0.8296
 0.8295 0.8296 0.8298 0.8302 0.8308 0.8314 0.8323 0.8332 0.8344 0.8356
 0.837  0.8385 0.8401 0.8419 0.8437 0.8456 0.8477 0.8498 0.8519 0.8542
 0.8564 0.8587 0.8611 0.8634 0.8657 0.8681 0.8703 0.8725 0.8747 0.8768
 0.8787 0.8806 0.8823 0.8838 0.8852 0.8864 0.8874 0.8881 0.8887 0.8891
 0.8892 0.8891 0.8887 0.8881 0.8874 0.8

In [2]:
import numpy as np
from scipy.optimize import least_squares
import pymc as pm

# -------------------------------
# Physikalische Konstanten
# -------------------------------
h = 6.62607015e-34  # Planck
c = 2.99792458e8    # Lichtgeschwindigkeit
kB = 1.380649e-23   # Boltzmann

# -------------------------------
# Planck-Funktion (modifiziert nach deinem Modell)
# -------------------------------
def planck_mod(lam_um, T):
    lam = lam_um * 1e-6  # µm -> m
    nu = c / lam
    num = 2*h*nu**3 / c**2
    denom = np.exp(h*nu/(kB*T)) - 1 + (h*nu/(kB*T))**2/(10*(kB*T)**2)
    return num / denom  # W·m⁻²·sr⁻¹·m⁻¹

# -------------------------------
# Resonanzverstärkung
# -------------------------------
def psi_resonance(lam_um):
    lam = lam_um * 1e-6
    nu = c / lam
    x = 2*np.pi*nu*1e-14
    return (np.sin(x)**2) / (x**2 + 1e-30)

# -------------------------------
# Emissivität aus Reflexion
# -------------------------------
def emissivity(lam_um):
    Rs = 0.1  # Beispiel: 10% Reflexion
    return 1 - (2/np.pi)*np.arcsin(np.sqrt(Rs))

# -------------------------------
# Vorwärtsmodell (ohne Umgebungsreflexion)
# -------------------------------
def forward_model(T, lam_um, eps, use_resonance=True):
    B = planck_mod(lam_um, T)
    if use_resonance:
        psi = psi_resonance(lam_um)
        return eps * B * psi
    else:
        return eps * B

# -------------------------------
# Beispiel-Daten
# -------------------------------
lam_um = np.linspace(8.0, 12.0, 80)
T_true = 300.0  # Kelvin
eps = emissivity(lam_um) * np.ones_like(lam_um)

S_true = forward_model(T_true, lam_um, eps)
rng = np.random.default_rng(42)
S_meas = S_true * (1.0 + 0.01*rng.standard_normal(S_true.shape))  # mit Rauschen

# ===================================================
# 1) Least-Squares mit Laplace-Approx für Unsicherheit
# ===================================================
def residual_T(T_vec):
    T = float(T_vec[0])
    model = forward_model(T, lam_um, eps)
    return S_meas - model

res = least_squares(residual_T, x0=[300.0], bounds=(200.0, 500.0), max_nfev=200)

T_est = float(res.x[0])
N = S_meas.size
p = 1
RSS = 2*res.cost
s2 = RSS / (N - p)

J = res.jac
JTJ_inv = np.linalg.inv(J.T @ J)
var_T = s2 * JTJ_inv[0,0]
std_T = np.sqrt(var_T)

print("==== Least Squares mit Laplace-Unsicherheit ====")
print(f"T_true = {T_true:.2f} K")
print(f"T_est  = {T_est:.2f} K ± {1.96*std_T:.2f} K (95% CI)")
print()

# ===================================================
# 2) Volle Bayes-Schätzung mit PyMC
# ===================================================
with pm.Model() as model:
    # Prior für T
    T = pm.Normal("T", mu=300.0, sigma=50.0)
    sigma = pm.HalfNormal("sigma", sigma=1e-8)

    mu = forward_model(T, lam_um, eps)

    # Likelihood (Gaussian Noise)
    y = pm.Normal("y", mu=mu, sigma=sigma, observed=S_meas)

    trace = pm.sample(2000, tune=1000, target_accept=0.9, chains=2, progressbar=False)

T_samples = trace.posterior["T"].values.ravel()
T_mean = np.mean(T_samples)
T_ci = np.percentile(T_samples, [2.5, 97.5])

print("==== Bayesian Estimation mit PyMC ====")
print(f"T ≈ {T_mean:.2f} K (95% CI: {T_ci[0]:.2f} .. {T_ci[1]:.2f} K)")


==== Least Squares mit Laplace-Unsicherheit ====
T_true = 300.00 K
T_est  = 300.00 K ± 0.12 K (95% CI)



Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [T, sigma]
Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 1 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


==== Bayesian Estimation mit PyMC ====
T ≈ 300.01 K (95% CI: 299.89 .. 300.14 K)


In [3]:
# !pip install pymc arviz numpy scipy

import numpy as np
import pymc as pm
import arviz as az
import aesara.tensor as at

# -------------------------------
# Physikalische Konstanten
# -------------------------------
h  = 6.62607015e-34   # Planck [J·s]
c  = 2.99792458e8     # Lichtgeschwindigkeit [m/s]
kB = 1.380649e-23     # Boltzmann [J/K]
PI = np.pi

# -------------------------------
# Numpy-Helfer (für Datensynthese)
# -------------------------------
def nu_from_lambda_np(lam_um):
    lam_m = lam_um * 1e-6
    return c / lam_m

def Bstar_nu_np(T, lam_um):
    """Modifizierte Planck-Funktion 𝓑*ν(T) (Frequenzform) in Numpy."""
    nu = nu_from_lambda_np(lam_um)
    x  = (h*nu)/(kB*T)
    denom = np.exp(x) - 1.0 + (x**2)/10.0
    return (2.0*h*nu**3)/(c**2) / denom  # W·m^-2·sr^-1·Hz^-1 (relative Skala ok)

def psi_lambda_np(lam_um, osc_const=1e-14):
    """Ψ(λ) ~ (sin(2πνC)/(2πνC))^2 in Numpy, numerisch stabil."""
    nu = nu_from_lambda_np(lam_um)
    x  = 2.0*PI*nu*osc_const
    y  = np.ones_like(x)
    nz = np.abs(x) > 1e-12
    y[nz] = (np.sin(x[nz])/x[nz])**2
    return y

def emissivity_from_Rs_np(Rs):
    """ε(λ) = 1 - (2/π) * arcsin( sqrt(Rs) )"""
    Rs = np.clip(Rs, 0.0, 1.0)
    return 1.0 - (2.0/PI)*np.arcsin(np.sqrt(Rs))

def forward_np(T, lam_um, eps, X=None, use_reflection=True):
    """Vorwärtsmodell: S(λ,T) = ε·𝓑*ν(T)·Ψ + (1-ε)·X (optional) – Numpy-Version."""
    S_emit = eps * Bstar_nu_np(T, lam_um) * psi_lambda_np(lam_um)
    if use_reflection and (X is not None):
        return S_emit + (1.0 - eps) * X
    else:
        return S_emit

# -------------------------------
# Aesara-Helfer (für PyMC-Graph)
# -------------------------------
def Bstar_nu_tt(T, lam_um_tt):
    """𝓑*ν(T) in Aesara."""
    lam_m = lam_um_tt * 1e-6
    nu    = c / lam_m
    x     = (h*nu)/(kB*T)
    denom = at.exp(x) - 1.0 + (x**2)/10.0
    return (2.0*h*nu**3)/(c**2) / denom

def psi_lambda_tt(lam_um_tt, osc_const=1e-14):
    """Ψ(λ) = (sin(2πνC)/(2πνC))^2 in Aesara, numerisch stabil."""
    lam_m = lam_um_tt * 1e-6
    nu    = c / lam_m
    x     = 2.0*PI*nu*osc_const
    # sin(x)/x stabilisieren:
    y     = at.sin(x) / (x + 1e-12)
    return y**2

def forward_tt(T, lam_um_tt, eps_tt, X_tt=None, use_reflection=True):
    """Vorwärtsmodell in Aesara: S = ε·𝓑*ν(T)·Ψ + (1-ε)·X (optional)."""
    S_emit = eps_tt * Bstar_nu_tt(T, lam_um_tt) * psi_lambda_tt(lam_um_tt)
    if use_reflection and (X_tt is not None):
        return S_emit + (1.0 - eps_tt) * X_tt
    else:
        return S_emit

# -------------------------------
# Beispiel: synthetische Daten erzeugen
# (Wenn du reale Daten hast, ersetze diesen Block einfach durch deine Arrays)
# -------------------------------
lam_um = np.linspace(8.0, 12.0, 81)                    # LWIR-Fenster
# Dummy-Reflexionskurve -> ε(λ)
Rs = 0.05 + 0.02*np.sin(2*np.pi*(lam_um-8.0)/4.0)
eps_true = emissivity_from_Rs_np(Rs)

# Umgebungsspektrum X(λ): optional
X = 1e-6 * (0.6 + 0.4*np.cos(2*np.pi*(lam_um-8.0)/4.0))

T_true = 300.0
use_reflection = True

# "Messung" mit leichtem Rauschen (hier gauss., für Student-t ist das ok)
rng = np.random.default_rng(123)
S_true = forward_np(T_true, lam_um, eps_true, X=X, use_reflection=use_reflection)
S_meas = S_true * (1.0 + 0.01*rng.standard_normal(S_true.shape))

# -------------------------------
# PyMC-Modell (Bayes, Heavy-Tail via Student-t)
# -------------------------------
with pm.Model() as model:
    # Statische Daten ins Modell einspeisen
    lam_data = pm.MutableData("lam_um", lam_um)
    eps_data = pm.MutableData("eps", eps_true)
    X_data   = pm.MutableData("X", X if use_reflection else np.zeros_like(lam_um))
    Y_obs    = pm.MutableData("S_meas", S_meas)

    # PRIORS
    # Temperatur: relativ breit
    T = pm.Normal("T", mu=300.0, sigma=60.0)

    # Emissivitäts-Kalibrierung (globaler Multiplikator ~1, nur wenn gewünscht)
    gamma_eps = pm.LogNormal("gamma_eps", mu=0.0, sigma=0.15)  # Median 1.0
    eps_eff = pm.math.clip(gamma_eps * eps_data, 1e-3, 0.999)

    # Reflexions-Skalierung (optional)
    if use_reflection:
        alpha_X = pm.LogNormal("alpha_X", mu=0.0, sigma=0.5)  # Median 1.0, flexibel
        X_eff   = alpha_X * X_data
    else:
        X_eff   = None

    # Vorwärtsmodell im Graph
    mu = forward_tt(T, lam_data, eps_eff, X_tt=X_eff, use_reflection=use_reflection)

    # Heavy-Tail Rauschen: Student-t mit df (nu) und Skala sigma
    # Tipp: setze sigma-Prior in Größenordnung deiner Messwerte
    sigma = pm.HalfNormal("sigma", sigma=np.std(S_meas)*0.5 + 1e-20)
    nu    = pm.Exponential("nu", lam=1/10)  # mittl. df ~10 (robust)

    y = pm.StudentT("y", nu=nu, mu=mu, sigma=sigma, observed=Y_obs)

    # Inferenz
    idata = pm.sample(
        draws=2500, tune=2000, chains=2, target_accept=0.95, progressbar=False
    )

# -------------------------------
# Ergebnisse zusammenfassen
# -------------------------------
print("\n=== Posterior-Zusammenfassung (wichtigste Parameter) ===")
print(az.summary(idata, var_names=["T", "gamma_eps", "alpha_X", "sigma", "nu"], round_to=4))

# 95%-Intervall für T
T_samples = idata.posterior["T"].values.ravel()
T_mean = np.mean(T_samples)
T_ci   = np.percentile(T_samples, [2.5, 97.5])
print(f"\nT ≈ {T_mean:.2f} K  (95% CI: {T_ci[0]:.2f} .. {T_ci[1]:.2f} K)")

# Posterior-predictive Check (optional)
with model:
    ppc = pm.sample_posterior_predictive(idata, var_names=["y"], progressbar=False)

y_pred = ppc["y"]  # Shape: [draws, n_points]
y_mean = y_pred.mean(axis=0)
rmse   = np.sqrt(np.mean((y_mean - S_meas)**2))
print(f"Posterior-predictive RMSE: {rmse:.3e}")


ModuleNotFoundError: No module named 'aesara'